In [ ]:
import requests, base64, zipfile, io, time, json, os
from pprint import pprint
jobs = []

In [ ]:
opts = {
        "data":{ 
            "lookup"           : "Escherichia coli",
            "PAM"              : "NGG",
            "Orientation"      : "3prime",
            "Guide_Length"     : 20,
            "Seed_Length"      : 10,
            "edit_dist"        : 6,
            "gene_density_len" : 10000,
            #"intspace"         : 400,
            "blast_org"        : "Escherichia coli MG1655 I,Escherichia coli MG1655 II"
        },
        "vcpus":16,
        "memory":15000,
        "timeout": 3600
       }

r = requests.post("https://5fakh91t4f.execute-api.us-east-2.amazonaws.com/default/COPIES-runner", json=opts)
post_data =json.loads(r.content.decode("utf-8"))
print("Started job with id: {}".format(post_data["id"]))
jobs.append(post_data["id"])

In [ ]:
print("Waiting for {} to complete (polling every 5s)...".format(post_data["id"]))
complete = 0
while complete < 100:
    r = requests.post("https://5fakh91t4f.execute-api.us-east-2.amazonaws.com/default/COPIES-runner",
                      json={"get_update":post_data["id"]})
    progress_data = json.loads(r.content.decode("utf-8"))
    complete = progress_data["percent_complete"]
    time.sleep(5)
    
r = requests.get(progress_data["download_url"])
os.makedirs("./{}".format(post_data["id"]))
zipfile.ZipFile(io.BytesIO(r.content)).extractall(path="./{}".format(post_data["id"]))

print("------------")
with open("./{}/output.txt".format(post_data["id"]), "r") as f:
    print(f.read())
print("------------")
with open("./{}/output_data.csv".format(post_data["id"]), "r") as f:
    print(f.read())

In [ ]:
# How to upload a custom organism
# Maximum filesize is 100MB

# request upload keys
r = requests.post("https://rbsoooozb6.execute-api.us-east-2.amazonaws.com/default/COPIES-customupload", json={"request" : "keys"})
key_data =json.loads(r.content.decode("utf-8"))

def upload_file( file_key, file_name):
    with open(file_name, "rb") as f:
        fields = file_key["fields"]
        fields["file"] = ("f", f, "application/octet-stream")
        r = requests.post(file_key["url"], files=fields)
        assert r.status_code == 204
    
upload_file(key_data["protein"], "GCF_000146045.2_R64_protein.faa")
upload_file(key_data["genome"], "GCF_000146045.2_R64_genomic.fna")
upload_file(key_data["feature_table"], "GCF_000146045.2_R64_feature_table.txt")

r = requests.post("https://rbsoooozb6.execute-api.us-east-2.amazonaws.com/default/COPIES-customupload", json={"request" : "validate", "upload_id" : key_data["upload_id"]})
print("file validation:")
print(json.loads(r.content.decode("utf-8")))